<h1> Cross Encoder Reranker

In [4]:
# Retrieval Ranking
# Role: Searches and ranks relevant documents from a large collection during the initial stage.
# Methods:
# - Sparse Retrieval: Uses traditional models like TF-IDF or BM25.
# - Dense Retrieval: Employs bi-encoders to separately embed queries and documents, computing similarity (e.g., cosine similarity).

# Cross Encoder Reranking
# Role: Re-ranks the top-k documents retrieved during the initial stage for better relevance.
# Mechanism:
# Combines the query and document as a single input to analyze their semantic relationship using self-attention.
# Outputs a relevance score for each query-document pair.


In [ ]:

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:

# Load Documents
documents = TextLoader("./data/appendix-keywords.txt").load()

# Text Splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

# Embedding Model Setup
embeddingsModel = HuggingFaceEmbeddings(
    model_name="sentence-transformers/msmarco-distilbert-dot-v5"
)

# Use FAISS to create an index from the text chunks, then set up a retriever to search the index with top-k results (k=10)
retriever = FAISS.from_documents(texts, embeddingsModel).as_retriever(
    search_kwargs={"k": 10}
)

# Query and Retrieve Results
query = "What is monetary policy like in Korea?"


# Embeds the query using the HuggingFaceEmbeddings model and performs a similarity search in the FAISS index using the query's embedding
docs = retriever.invoke(query)

pretty_print_docs(docs)

# So far, The query and document embeddings are computed independently using the Hugging Face model (msmarco-distilbert-dot-v5).
# Now we The query and document are processed together as a single input through a model by cross encoder

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# Cross Encoder Initialization
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# The CrossEncoderReranker is initialized with the model and configured to select the top 3 most relevant documents (top_n=3)
compressor = CrossEncoderReranker(model=model, top_n=3)

# ContextualCompressionRetriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# Query Execution
compressed_docs = compression_retriever.invoke("What is monetary policy like in Korea?")

pretty_print_docs(compressed_docs)